# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [7]:
import praw
import pandas as pd
reddit = praw.Reddit(client_id = "", client_secret= "", user_agent= "")

We can now access the subreddit, extract all the posts in it and check which other subreddit reposted that posts and how much:

In [5]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = ["conspiracy"]

for i in reddit.subreddit("conspiracy").hot(limit=5000): #recupera i 5000 post top di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + il permalink


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        if repost.subreddit_id not in id_to_avoid:
            
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1][0] +=1
            else:
                conspiracy_dict[subreddit_url]=[[],[1]]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_top_url.csv',index=False)




Here we can also estract the posts with the same permalink:

In [2]:
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = ["conspiracy"]

for i in reddit.subreddit("conspiracy").top(limit=5000):
    post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]):
        if repost.subreddit_id not in id_to_avoid:
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1][0] +=1
            else:
                conspiracy_dict[subreddit_url]=[[], [1]]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_top.csv',index=False)

# Now we have the CSV

In [8]:
df1 = pd.read_csv('results/conspiracy_hot.csv')
id_to_analyse = []
for column in df1.columns:
    value = df1[column][1][1:-1]
    if int(value) >= 5:
        ind = column.index('/r/')
        id = column[ind+3:]
        id_to_analyse.append(id)

for subreddit in id_to_analyse:
    post_list=list()
    subreddit_list = list()
    conspiracy_dict=dict() 
    for i in reddit.subreddit(subreddit).top(limit=5000):
        post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


    for post in post_list:
        for repost in reddit.subreddit('all').search('url:'+post[2]):
            if repost.subreddit_id != subreddit: #cosa facciamo?
                subreddit_url = str(repost.subreddit)
                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                if subreddit_url in conspiracy_dict.keys():
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                    conspiracy_dict[subreddit_url][1][0] +=1
                else:
                    conspiracy_dict[subreddit_url]=[[],[1]]
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
    df = pd.DataFrame(conspiracy_dict)
    df.to_csv(r'results/'+subreddit+'.csv')